## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.tipo = tipo
        self._precio = 0  
        self._cantidad = 0
        self.precio = precio  
        self.cantidad = cantidad

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, prec):
        if prec >= 10 and prec <=10000:
            self._precio = prec

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, cant):
        if cant >= 0 and cant <=1000:
            self._cantidad = cant

    def valorTotal(self):
        return self._cantidad * self._precio 

In [2]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos

    def esAplicable(self, producto):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

class OfertaDescuento(Oferta):
    def __init__(self, descuento, descripcion='Descuento 10%', **kwargs):
        super().__init__(descripcion, **kwargs)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            return  self.descuento / 100 * producto.precio * cantidad
        return 0

    def aplicar(self, producto, cantidad):
         return  producto.precio * cantidad - self.calcularDescuento(producto, cantidad)
         

class Oferta2x1(Oferta):
    def __init__(self, descripcion ='Oferta 2x1', **kwargs):
        super().__init__(descripcion, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            return  producto.precio * (cantidad // 2) 
        return 0

    def aplicar(self, producto, cantidad):
        return producto.precio * cantidad - self.calcularDescuento(producto, cantidad)


In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, mode='r') as file:
            reader = csv.reader(file)
            lineas_saltadas = 0
            for index, row in enumerate(reader):
                if index == 0:
                    lineas_saltadas += 1
                else:
                    if len(row) == 5:
                        codigo, nombre, precio, tipo, cantidad = row
                        precio_convertido = int(precio)
                        cantidad_convertida = int(cantidad)
                        nuevo_producto = Producto(codigo, nombre, precio_convertido, tipo, cantidad_convertida)
                        catalogo.productos.append(nuevo_producto)
        return catalogo

    def guardar(self, archivo):
        with open(archivo, mode='w', newline='') as file:
            writer = csv.writer(file)
            headers = ['codigo', 'nombre', 'precio', 'tipo', 'cantidad']
            writer.writerow(headers)
            for producto in self.productos:
                fila_producto = [producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad]
                writer.writerow(fila_producto)

    def agregar(self, *producto):
        for prod in producto:
            self.productos.append(prod)

    def buscar(self, codigo):
        encontrado = None
        for producto in self.productos:
            if producto.codigo == codigo:
                encontrado = producto
                break
        return encontrado

    def registrarOferta(self, oferta):
        if oferta not in self.ofertas:
            self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        oferta_encontrada = None
        for oferta in self.ofertas:
            if oferta.esAplicable(producto):
                oferta_encontrada = oferta
                break
        return oferta_encontrada

    def calcularDescuento(self, producto, cantidad):
        oferta_aplicable = self.buscarOferta(producto, cantidad)
        if oferta_aplicable is not None:
            descuento = oferta_aplicable.calcularDescuento(producto, cantidad)
        else:
            descuento = 0
        return descuento

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        total_unidades = 0
        for producto in self.productos:
            total_unidades += producto.cantidad
        return total_unidades

    @property
    def valorTotal(self):
        valor_total = 0
        for producto in self.productos:
            valor_total += producto.valorTotal()
        return valor_total

    def vender(self, producto, cantidad):
        producto_en_catalogo = self.buscar(producto.codigo)
        if producto_en_catalogo is not None:
            if producto_en_catalogo.cantidad >= cantidad:
                producto_en_catalogo.cantidad -= cantidad

    def informe(self):
        tipos = set()
        for producto in self.productos:
            tipos.add(producto.tipo)

        informe = "Cantidad de productos: {}\n".format(self.cantidadProductos)
        informe += "Cantidad de unidades: {}\n".format(self.cantidadUnidades)
        if self.cantidadUnidades > 0:
            promedio = self.valorTotal / self.cantidadUnidades
            informe += "Precio Promedio: {:.2f}\n".format(promedio)
        informe += "Valor total: {}\n".format(self.valorTotal)
        informe += "Tipos de productos: {}\n".format(", ".join(tipos))
        informe += "Ofertas:\n"
        
        for oferta in self.ofertas:
            informe += "- {}\n".format(oferta.descripcion)
        
        return informe


In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:

class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = ""
        self._cuit = ""
        self.nombre = nombre 
        self.cuit = cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if str(valor) and (len(valor) >= 1 and len(valor)<= 100):
            self._nombre = valor
        elif self._nombre == "":
            self._nombre = "Nombre inválido"

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        if self._es_cuit_valido(valor):
            self._cuit = valor
        elif self._cuit == "":
            self._cuit = "CUIT inválido"

    def _es_cuit_valido(self, cuit):
        if str(cuit) and '-' in cuit:
            partes = cuit.split('-')
            return len(partes) == 3 and all(self._parte_valida(p, i) for i, p in enumerate(partes))
        return False

    def _parte_valida(self, parte, indice):
        longitudes = [2, 8, 1]
        return parte.isdigit() and len(parte) == longitudes[indice]


In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [9]:
from datetime import datetime

class Factura:
    _ultimo_numero = 0  

    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        fecha_actual = datetime.now()
        self.fecha = fecha_actual.strftime("%Y-%m-%d %H:%M:%S")     
        self.cliente = cliente      
        self.catalogo = catalogo      
        self.items = {}

    @classmethod
    def ultimaFactura(cls, numero):
        numero_actualizado = numero
        cls._ultimo_numero = numero_actualizado

    @classmethod
    def nuevoNumero(cls):
        numero_anterior = cls._ultimo_numero
        nuevo_numero = numero_anterior + 1
        cls._ultimo_numero = nuevo_numero
        return nuevo_numero

    @property
    def cantidadProductos(self):
        total_productos = len(self.items)
        return total_productos

    @property
    def cantidadUnidades(self):
        total_unidades = sum(self.items.values())
        return total_unidades

    @property
    def subtotal(self):
        subtotal_calculado = 0
        for producto, cantidad in self.items.items():
            subtotal_producto = producto.precio * cantidad
            subtotal_calculado += subtotal_producto
        return subtotal_calculado

    @property
    def descuentos(self):
        total_descuentos_calculado = 0
        for producto, cantidad in self.items.items():
            descuento_producto = self.catalogo.calcularDescuento(producto, cantidad)
            total_descuentos_calculado += descuento_producto
        return total_descuentos_calculado

    @property
    def total(self):
        subtotal_actual = self.subtotal
        descuentos_actuales = self.descuentos
        total_final = subtotal_actual - descuentos_actuales
        return total_final

    def agregar(self, producto, cantidad):
        producto_en_catalogo = self.catalogo.buscar(producto.codigo)
        if producto_en_catalogo is not None:
            stock_disponible = producto_en_catalogo.cantidad
            if stock_disponible >= cantidad:
                self.catalogo.vender(producto_en_catalogo, cantidad)
                if producto_en_catalogo in self.items:
                    cantidad_existente = self.items[producto_en_catalogo]
                    nueva_cantidad = cantidad_existente + cantidad
                    self.items[producto_en_catalogo] = nueva_cantidad
                else:
                    self.items[producto_en_catalogo] = cantidad
            else:
                print(f"No hay suficiente stock de {producto_en_catalogo.nombre}.")
        else:
            print(f"El producto con código {producto.codigo} no existe en el catálogo.")

    def imprimir(self):
        impresion = ""
        impresion += f"Factura: {self.numero}\n"
        impresion += f"Fecha  : {self.fecha}\n"
        impresion += f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"

        for producto, cantidad in self.items.items():
            subtotal_producto = producto.precio * cantidad
            descuento_producto = self.catalogo.calcularDescuento(producto, cantidad)
            impresion += f"- {cantidad} {producto.nombre}            x ${producto.precio} = ${subtotal_producto}\n"
            if descuento_producto > 0:
                oferta_producto = self.catalogo.buscarOferta(producto, cantidad)
                descripcion_oferta = oferta_producto.descripcion if oferta_producto else "Descuento"
                impresion += f"      {descripcion_oferta}                             - ${descuento_producto}\n"

        subtotal_factura = self.subtotal
        descuentos_factura = self.descuentos
        total_factura = self.total

        impresion += "\n"
        impresion += f"{' ' * 49}Subtotal:   ${subtotal_factura:.2f}\n"
        impresion += f"{' ' * 49}Descuentos: ${descuentos_factura:.2f}\n"
        impresion += f"{' ' * 49}-----------------------\n"
        impresion += f"{' ' * 49}TOTAL:      ${total_factura:.2f}\n"

        return impresion


In [10]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
